In [42]:
#For the webscraper
import requests, json
from time import sleep
from bs4 import BeautifulSoup

start_year_min = 2000
end_year_min = 2010

In [118]:
def get_3pt_shooting_data(player_url):
    
    r = requests.get(player_url, verify=False)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    ## Get the player name
    meta_section = soup.find_all('div', {'id': 'meta'})
    if len(meta_section) == 0:
        return
    player_name = meta_section[0].find_all('span')[0].contents[0]
    
    totals_table_list = soup.find_all('table', {'id': 'totals'})
    ## If empty, then return empty
    if len(totals_table_list) == 0:
        return
    
    ## Get the rows of the table
    totals_table = totals_table_list[0]
    rows = totals_table.find_all('tr')
    
    ## For each row, get the year, fg3, fg3a, and fg3_pct
    row_data = []
    for r in rows:
        tds = r.find_all('td')
        if (len(tds)==0):
            continue
            
        if (len([x for x in tds if 'data-stat="age"' in str(x)][0].contents)==0):
            continue
            
        age = [x for x in tds if 'data-stat="age"' in str(x)][0].contents[0] #r.find_all('td', {'data-stat':'age'})
        fg3 = [x for x in tds if 'data-stat="fg3"' in str(x)][0].contents[0] #r.find_all('td', {'data-stat':'fg3'})
        fg3a = [x for x in tds if 'data-stat="fg3a"' in str(x)][0].contents[0] #r.find_all('td', {'data-stat':'fg3a'})
        if len([x for x in tds if 'data-stat="fg3_pct"' in str(x)][0].contents)==0:
            fg3_pct = '0'
        else:
            fg3_pct = [x for x in tds if 'data-stat="fg3_pct"' in str(x)][0].contents[0] #r.find_all('td', {'data-stat':'fg3_pct'})
        row_dict = {'age': age, 'fg3': fg3, 'fg3a': fg3a, 'fg3_pct': fg3_pct}
        row_data.append(row_dict)
    
    return_data = {
        'name': player_name,
        'url': player_url,
        'row_data': row_data,
    }
    return return_data

In [130]:
def get_player_hrefs(base_url, alph):
    
    r = requests.get(base_url + '/' + alph + '/', verify=False)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    data_rows = soup.find_all('tr')
    player_rows = [x for x in data_rows if 'data-stat="player"' in str(x)]
    
    player_href_links = []
    for r in player_rows:
        cells_th = r.find_all('th')
        cells_td = r.find_all('td')

        ## Filter by start/end years
        start_year, end_year = None, None
        start_year_html = [x for x in cells_td if 'data-stat="year_min"' in str(x)]
        end_year_html = [x for x in cells_td if 'data-stat="year_max"' in str(x)]
        if (len(start_year_html)>0):
            start_year = start_year_html[0].contents[0]
        if (len(end_year_html)>0):
            end_year = end_year_html[0].contents[0]

        #print (start_year, end_year)
        if start_year and end_year:
            if int(start_year) < start_year_min:
                continue
            if int(end_year) < end_year_min:
                continue

        ## try to get the href link
        player_link_results = cells_th[0].find_all('a', href=True)
        if len(player_link_results) > 0:
            player_href_links.append(player_link_results[0].get('href'))
            
    return player_href_links

In [115]:
base_url = 'https://www.basketball-reference.com'
url = base_url + '/players/a/'
r = requests.get(url, verify=False)
soup = BeautifulSoup(r.text, 'html.parser')

#data_rows = soup.find_all('tr', {'class': 'data-row'})
#data_rows = soup.find_all('tr', {'data-row': True})
data_rows = soup.find_all('tr')
#print (data_rows)

#player_cells = data_rows.find_all('th', {'data-stat': "player"})

## Filter data-rows down to the ones with data-stat='player'
player_rows = [x for x in data_rows if 'data-stat="player"' in str(x)]

#for r in player_rows:
#    print (r)

## Get the hrefs for the player links
player_href_links = []
for r in player_rows:
    cells_th = r.find_all('th')
    cells_td = r.find_all('td')
    #print (cells_th)
    #print (cells_td)
    
    ## Filter by start/end years
    start_year, end_year = None, None
    start_year_html = [x for x in cells_td if 'data-stat="year_min"' in str(x)]
    end_year_html = [x for x in cells_td if 'data-stat="year_max"' in str(x)]
    if (len(start_year_html)>0):
        start_year = start_year_html[0].contents[0]
    if (len(end_year_html)>0):
        end_year = end_year_html[0].contents[0]
    
    #print (start_year, end_year)
    if start_year and end_year:
        if int(start_year) < start_year_min:
            continue
        if int(end_year) < end_year_min:
            continue
    
    ## try to get the href link
    player_link_results = cells_th[0].find_all('a', href=True)
    if len(player_link_results) > 0:
        #print (player_link_results[0].get('href'))
        player_href_links.append(player_link_results[0].get('href'))
        #print (start_year, end_year, player_link_results[0].get('href'))
        
    
    

C:\Users\572784\Miniconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.basketball-reference.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [142]:
#output = get_3pt_shooting_data('https://www.basketball-reference.com'+'/players/a/abrinal01.html')
#print (player_href_links)

all_player_data = []
alphs = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
for alph in alphs:
    
    player_href_links = get_player_hrefs('https://www.basketball-reference.com/players', alph)
    
    for player_url in player_href_links:
        print (alph, player_url)
        try:
            player_data = get_3pt_shooting_data(base_url + player_url)
            if ({'name','url','row_data'}==(set(player_data.keys()))):
                all_player_data.append(player_data)
        except Exception as e:
            print ('ERROR:', player_url)
            print (e)

        #print (player_url, player_data)

C:\Users\572784\Miniconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.basketball-reference.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


a /players/a/abrinal01.html


C:\Users\572784\Miniconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.basketball-reference.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


a /players/a/achiupr01.html


C:\Users\572784\Miniconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.basketball-reference.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


...

C:\Users\572784\Miniconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.basketball-reference.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


z /players/z/zizican01.html


C:\Users\572784\Miniconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.basketball-reference.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


z /players/z/zubaciv01.html


C:\Users\572784\Miniconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.basketball-reference.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [143]:
print (all_player_data)

[{'name': 'Álex Abrines', 'url': 'https://www.basketball-reference.com/players/a/abrinal01.html', 'row_data': [{'age': '23', 'fg3': '94', 'fg3a': '247', 'fg3_pct': '.381'}, {'age': '24', 'fg3': '84', 'fg3a': '221', 'fg3_pct': '.380'}, {'age': '25', 'fg3': '41', 'fg3a': '127', 'fg3_pct': '.323'}]}, {'name': 'Precious Achiuwa', 'url': 'https://www.basketball-reference.com/players/a/achiupr01.html', 'row_data': [{'age': '21', 'fg3': '0', 'fg3a': '1', 'fg3_pct': '.000'}, {'age': '22', 'fg3': '56', 'fg3a': '156', 'fg3_pct': '.359'}]}, {'name': 'Quincy Acy', 'url': 'https://www.basketball-reference.com/players/a/acyqu01.html', 'row_data': [{'age': '22', 'fg3': '1', 'fg3a': '2', 'fg3_pct': '.500'}, {'age': '23', 'fg3': '4', 'fg3a': '15', 'fg3_pct': '.267'}, {'age': '23', 'fg3': '2', 'fg3a': '5', 'fg3_pct': '.400'}, {'age': '23', 'fg3': '2', 'fg3a': '10', 'fg3_pct': '.200'}, {'age': '24', 'fg3': '18', 'fg3a': '60', 'fg3_pct': '.300'}, {'age': '25', 'fg3': '19', 'fg3a': '49', 'fg3_pct': '.388'}

In [144]:
## Copy the data over here just so we don't muck it up
all_player_data_copy = [x for x in all_player_data]

In [145]:
## Need to go through and covnert bs4 type objects to strings
for x in all_player_data_copy:
    #for k in x.keys():
    #    x[k] = str(x[k])
    x['name'] = str(x['name'])
    x['url'] = str(x['url'])
    for row in x['row_data']:
        row['age'] = str(row['age'])
        row['fg3'] = str(row['fg3'])
        row['fg3a'] = str(row['fg3a'])
        row['fg3_pct'] = str(row['fg3_pct'])

In [146]:
import json

with open('./nba_3pt_progressions_all_players.json', 'w') as f:
    json.dump(all_player_data_copy, f)
    f.close()